In [3]:
from skimage.io import imread
import numpy as np

data = imread('./ee19f416b1735c6ec5fc2ff3c1524a761032d001/450_accum16_20200212_163253.png')
flat = data.flatten()
# Use a listing of the unique values to guide their counting and then sort on those counters to find the most efficient pixel
# map for this image.  We must store the pixel map in image metadata for this to work, but this overheard comes nowehre near
# undermining the cost savings for storing the main body.
uniques = np.unique(flat)
np.sort(uniques)
size = len(uniques)
uniques.resize(size + 1)
uniques[size] = np.max(uniques) + 5
ht = np.histogram(flat, bins=uniques, density=False)
values = np.array([(ii[0],0-ii[1]) for ii in zip(ht[1], ht[0])], 
                  dtype=[("value", int), ("freq", int)])
values.sort(order="freq")
encoding_map = [ii[0] for ii in values]
# MySQL varint algorithm has worked best thus far and it uses 8 bits for the first 240 values to receive varint encoding.
# Let's use that to predict how much of the image this will let us store efficiently
optimalPixels = np.sum([(0 - ii[1]) for ii in values[0:240]])
print(f"Optimal encoding for {optimalPixels} out of {len(flat)} total pixel count")


Optimal encoding for 2888549 out of 3048192 total pixel count


In [23]:
2888549 * 100.0 / 3048192 

94.76269867514907

For this sample input a carefully selected pixel encoding map reaches 94.7% of the total content with an eight bit pixel.  The remaining vlaues are going to fix into two pixels and not be any worse than they already are today.